# Efficient Frontier in Python

## References
* [`scipy minimize with constraints`](https://stackoverflow.com/questions/20075714/scipy-minimize-with-constraints)
## Steps to implement Modern Portfolio Theory
![](mean_variance_optimization.PNG)
* 1. Get the maximized Sharpe Ratio Portfolio
    * get corresponding return (return_start) and variance
* 2. Get the minimized Volatility Portfolio
    * get corresponding return (return_end) and variance
* 3. For the range of returns (return_start, return_end):
    * get corresponding minimized std
* 4. Draw the line of (range of returns, minimized stds)

In [141]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import scipy.optimize as sc

# import pandas_datareader as pdr
import yfinance as yf

import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [155]:
def get_mean_var(tickers:list, start_date: dt.datetime, end_date = dt.datetime):
    df_stocks = yf.download(tickers=tickers, start=start_date, end=end_date)['Close']
    returns = df_stocks.pct_change()
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    return mean_returns, cov_matrix
    # return df_stocks

def portfolio_performance(weights:np.array, mean_returns:np.array, cov_matrix:np.array):
    returns = np.sum(np.dot(weights, mean_returns))*252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))*252)
    return returns, std

def portfolio_return(weights:np.array, mean_returns:np.array, cov_matrix:np.array):
    return portfolio_performance(
        weights, mean_returns, cov_matrix)[0]
    
def portfolio_variance(weights:np.array, mean_returns:np.array, cov_matrix:np.array):
    return portfolio_performance(
        weights, mean_returns, cov_matrix)[1]

def negative_sr(weights, mean_returns, cov_matrix, risk_free_rate = 0):
    """get the negative sharpe ratio with different combination of weights
    
    Keyword arguments:
    weights -- the first argument is what we need to optimize over later using scipy
    
    Return: minimized negative sharpe ratio
    """
    pReturns, pStd = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(pReturns - risk_free_rate)/pStd

def maximize_sr(
    mean_returns, 
    cov_matrix, 
    risk_free_rate = 0, 
    # constraint_set = (0, 1)
):
    """minimize the negative sharpe ratio by altering the weights of the portfolio so that we get the maximized sharpe ratio
    we want the highest return for a given volatility
    """
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    linear_constraint = sc.LinearConstraint(np.ones((num_assets,), dtype=int),1,1)
    bounds = sc.Bounds(0, 1)
    results = sc.minimize(
        negative_sr, 
        num_assets*[1.0/num_assets], 
        args = args,
        method = 'SLSQP',
        bounds = bounds,
        constraints = [linear_constraint]
    )
    return results

def minimize_vol(
    mean_returns, 
    cov_matrix,
):
    """minimize the variance by altering the weights of the portfolio
    """
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    linear_constraint = sc.LinearConstraint(np.ones((num_assets,), dtype=int),1,1)
    bounds = sc.Bounds(0, 1)
    results = sc.minimize(
        portfolio_variance, 
        num_assets*[1.0/num_assets], 
        args = args,
        method = 'SLSQP',
        bounds = bounds,
        constraints = [linear_constraint]
    )
    return results

def efficient_frontier(mean_returns, cov_matrix, return_target):
    """for each return_target, we want to optimise the portfolio for min variance
    """
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    linear_constraint_weights = sc.LinearConstraint(
        np.ones((num_assets,), dtype=int),1,1
    )
    linear_constraint_return = {
        'type':'eq',
        'fun':lambda x:portfolio_return(x, mean_returns, cov_matrix) - return_target
    }
    bounds = sc.Bounds(0, 1)
    results = sc.minimize(
        portfolio_variance, 
        num_assets*[1.0/num_assets], 
        args = args,
        method = 'SLSQP',
        bounds = bounds,
        constraints = [
            linear_constraint_weights, # weights 加起来是1
            linear_constraint_return # return必须是给定target return
        ]
    )
    return results

def calculated_results(mean_returns, cov_matrix, risk_free_rate = 0):
    """read in mean, cov matrix, and other financial info
    
    Return: Max SR, Min Volatility, efficient frontier
    """
    
    # maximized portfolio sharpe ratio
    max_sr_portfolio = maximize_sr(
        mean_returns, cov_matrix, risk_free_rate = risk_free_rate
    )
    max_sr_returns, max_sr_std = portfolio_performance(
        max_sr_portfolio['x'], 
        mean_returns,
        cov_matrix
    )
    max_sr_allocation = pd.DataFrame(
        max_sr_portfolio['x'], 
        index = mean_returns.index,
        columns=["allocation"]
    )
    
    # minimized portfolio variance
    min_vol_portfolio = minimize_vol(
        mean_returns, cov_matrix
    )
    min_vol_returns, min_vol_std = portfolio_performance(
        min_vol_portfolio['x'], 
        mean_returns,
        cov_matrix
    )
    min_vol_allocation = pd.DataFrame(
        min_vol_portfolio['x'], 
        index = mean_returns.index,
        columns=["allocation"]
    )
    
    # construct the efficient frontier
    target_returns = np.linspace(min_vol_returns, max_sr_returns, 20)
    # 利用map函数可以进行运算加速
    efficient_list = map(
        lambda x: efficient_frontier(mean_returns, cov_matrix, x)['fun'], 
        target_returns
    )
    return (
        max_sr_returns, 
        max_sr_std, 
        max_sr_allocation, 
        min_vol_returns, 
        min_vol_std, 
        min_vol_allocation,
        target_returns,
        efficient_list
    )
    
def efficient_frontier_plot(        
    max_sr_returns, 
    max_sr_std, 
    min_vol_returns, 
    min_vol_std, 
    target_returns,
    efficient_list
):
    # Max Sharpe Ratio
    MaxSharpeRatio = go.Scatter(
        name = "Maximum Sharpe Ratio",
        mode = "markers",
        x = [max_sr_std],
        y = [max_sr_returns],
        marker = dict(
            color = 'red',
            size = 14,
            line = dict(width=3, color='black')
        )
    )
    
    # Min Vol
    MinVol = go.Scatter(
        name = "Maximum Volatility",
        mode = "markers",
        x = [min_vol_std],
        y = [min_vol_returns],
        marker = dict(
            color = 'green',
            size = 14,
            line = dict(width=3, color='black')
        )
    )
    
    # Efficient Frontier
    EFF = go.Scatter(
        name = "Efficient Frontier",
        mode = "lines",
        x = list(efficient_list),
        y = target_returns,
        marker = dict(
            color = 'green',
            size = 14,
            line = dict(width=3, color='black')
        )
    )
    
    data = [MaxSharpeRatio, MinVol, EFF]
    layout = go.Layout(
        title = 'Portfolio Optimisation with Efficient Frontier',
        yaxis = dict(title = 'Annulised Return (%)'),
        xaxis = dict(title = 'Annulised Volatility (%)'),
        showlegend = True,
        legend = dict(
            x = 0.75,
            y = 0,
            traceorder = 'normal',
            bgcolor = '#E2E2E2',
            bordercolor = 'black',
            borderwidth = 2
        ),
        width = 800,
        height = 600,
    )
    
    fig = go.Figure(data = data, layout = layout)
    
    return fig.show()

In [150]:
tickers = ["TSLA", "NVDA", "TQQQ", "TNA", "ARKK", "PDD", "META", "ISRG"]
# tickers = ["BABA", "DIS", "SE"]
start_date = dt.datetime(2015, 1, 1)
end_date = dt.datetime.now()

In [156]:
mean_returns, cov_matrix = get_mean_var(
    tickers=tickers, 
    start_date=start_date,
    end_date=end_date
)

[*********************100%%**********************]  8 of 8 completed


In [157]:
# mean_returns
# efficient_frontier(mean_returns, cov_matrix, 0.01)

In [158]:
(
    max_sr_returns, 
    max_sr_std, 
    max_sr_allocation, 
    min_vol_returns, 
    min_vol_std, 
    min_vol_allocation,
    target_returns,
    efficient_list
) = calculated_results(mean_returns, cov_matrix, risk_free_rate = 0)

In [159]:
# list(efficient_list)

In [160]:
efficient_frontier_plot(        
    max_sr_returns, 
    max_sr_std, 
    min_vol_returns, 
    min_vol_std, 
    target_returns,
    efficient_list
)

In [162]:
max_sr_allocation.apply(lambda x: round(x*100, 2))

,allocation
ARKK,0.00
ISRG,0.00
META,0.00
NVDA,71.74
PDD,10.22
TNA,0.00
TQQQ,0.00
TSLA,18.04
